# Model building
### Get clean features

In [ ]:
# source eda file
%run eda_lujain.ipynb

### Prerequisites

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor


## Data splitting
Split the data into training, testing and validation sets

In [14]:
X.shape

(17994, 14)

In [15]:
Y.shape

(17994,)

In [16]:
X_train,X_test,y_train,y_test = train_test_split(X, Y, train_size=0.7, random_state=123)

## Model Training

### XGBRegressor

In [ ]:
xGBR = XGBRegressor().fit(X_train,y_train)
predicted = xGBR.predict(X_test)


### XGBRegressor prediction accuracy

In [ ]:
score = np.sqrt(mean_squared_error(y_test, predicted))

### Random Forest Regressor

In [ ]:
RFR = RandomForestRegressor().fit(X_train,y_train)
predicted = RFR.predict(X_test)

### Random Forest Regressor predicion accuracy

In [ ]:
score = np.sqrt(mean_squared_error(y_test, predicted))

## Hyperparametere tuning

## Predictive accuracy